# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg


spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)


# Question 1

Which page did user id "" (empty string) NOT visit?

In [15]:
blank_pages = user_log.filter(user_log.userId=="").select(col("page")).alias("blank_pages").dropDuplicates()
all_pages = user_log.select('page').dropDuplicates()
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Downgrade
Save Settings
Upgrade
Logout
Submit Downgrade
NextSong
Submit Upgrade
Error
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


Perhaps it represents users who have not signed up yet or who are signed out and are about to log in.

# Question 3

How many female users do we have in the data set?

In [19]:
female_count = user_log.filter(user_log.gender == 'F').select("userId", "gender").dropDuplicates().count()
female_count

462

# Question 4

How many songs were played from the most played artist?

In [21]:
user_log.select("Artist").groupby("Artist")\
.agg({"Artist":"count"})\
.withColumnRenamed('count(Artist)', 'Artistcount')\
.sort(desc("Artistcount"))\
.show(1)


+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [26]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

